Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [16]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
import os
data_root = 'E:/MYGIT/DataSources/notMNIST'
pickle_file = os.path.join(data_root, 'notMNIST_remove_oevrlap.pickle')
pickle_file_1 = os.path.join(data_root, 'notMNIST.pickle')


with open(pickle_file_1, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
a = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
b = a.reshape(-1,2,2,1)
print(a.shape)
print(b.shape)

(2, 2, 2)
(2, 2, 2, 1)


In [9]:
print(a)
print(b)

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
[[[[1]
   [2]]

  [[3]
   [4]]]


 [[[5]
   [6]]

  [[7]
   [8]]]]


In [106]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [10]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
#VAlID参数时，窗口超出时，直接舍弃，SAME是超出部分补0
#input1 = tf.Variable(tf.reshape(tf.range(64),(1,8,8,1)))
input1 = tf.Variable(tf.ones([1,224,224,1]))#num_inputs, input_size, input_size, num_channels
filter1 = tf.Variable(tf.ones([11,11,1,1]))#kernel_size, kernel_size, num_channels, depth
op = tf.nn.conv2d(input1, filter1, [1,1,1,1], padding='SAME')
#第二个参数是window参数，第三个参数是stride参数
#op = tf.nn.max_pool(input1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

In [15]:
with tf.Session() as ses:
    ses.run(tf.global_variables_initializer())
    res = ses.run(op)
    print(res.shape)
    output = tf.reshape(res, [res.shape[1],res.shape[1]])
    #print(ses.run(output))    

(1, 224, 224, 1)


In [52]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    #patch_size核函数大小，depth表示核函数个数，代表会出现多少个卷积特征图像;
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        #卷积操作，strides[1]，strides[2]决定卷积核在输入图像in_hight，in_width方向的滑动步长
        conv = tf.nn.conv2d(data,layer1_weights, strides = [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits= logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
tf.nn.conv2d??

In [54]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.236959
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.417765
Minibatch accuracy: 56.2%
Validation accuracy: 69.3%
Minibatch loss at step 200: 1.004908
Minibatch accuracy: 81.2%
Validation accuracy: 76.7%
Minibatch loss at step 300: 0.527054
Minibatch accuracy: 87.5%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.755008
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.121012
Minibatch accuracy: 100.0%
Validation accuracy: 81.7%
Minibatch loss at step 600: 0.390446
Minibatch accuracy: 93.8%
Validation accuracy: 82.7%
Minibatch loss at step 700: 0.390476
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 800: 0.719820
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 900: 0.487813
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 0.550497
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [93]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        bias1 = tf.nn.relu(conv1 + layer1_biases)    
        pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        bias2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [94]:
num_steps = 1001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.395503
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.814555
Minibatch accuracy: 31.2%
Validation accuracy: 44.5%
Minibatch loss at step 200: 0.865961
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.444393
Minibatch accuracy: 93.8%
Validation accuracy: 77.4%
Minibatch loss at step 400: 0.700073
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.116253
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Minibatch loss at step 600: 0.543968
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 700: 0.453158
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 800: 0.557253
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Minibatch loss at step 900: 0.526162
Minibatch accuracy: 93.8%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.492508
Minibatch accuracy: 81.2%
Validation accuracy: 85.0

Minibatch loss at step 9000: 0.160583
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 9100: 0.169779
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 9200: 0.326287
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 9300: 0.351800
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 9400: 0.502658
Minibatch accuracy: 75.0%
Validation accuracy: 88.8%
Minibatch loss at step 9500: 0.368935
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 9600: 0.852087
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 9700: 0.455357
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 9800: 0.342029
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 9900: 0.547593
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 10000: 0.546381
Minibatch accuracy: 81.2%
Validation accuracy: 89.3

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The CNN below is loosely inspired by the LeNet5 architecture.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24   (28-5)/1 + 1 = 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        # C3 input 12 x 12  (24-2)/2+1 = 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8   (12-5)/1 + 1 = 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F6 input 4 x 4  (8-2)/2+1 =4
        
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.432403
Minibatch accuracy: 18.8%
Validation accuracy: 11.3%
Minibatch loss at step 50: 1.741629
Minibatch accuracy: 43.8%
Validation accuracy: 46.3%
Minibatch loss at step 100: 1.296582
Minibatch accuracy: 56.2%
Validation accuracy: 62.8%
Minibatch loss at step 150: 0.774304
Minibatch accuracy: 75.0%
Validation accuracy: 69.6%
Minibatch loss at step 200: 1.033114
Minibatch accuracy: 68.8%
Validation accuracy: 70.8%
Minibatch loss at step 250: 1.244565
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.575214
Minibatch accuracy: 87.5%
Validation accuracy: 76.5%
Minibatch loss at step 350: 0.622662
Minibatch accuracy: 87.5%
Validation accuracy: 75.1%
Minibatch loss at step 400: 0.430026
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%
Minibatch loss at step 450: 0.963384
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 500: 0.817511
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
M

The accuracy is good, but not as good as the 3-layer network from the previous assignment.

The next version of the net uses dropout and learning rate decay:

In [96]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
    
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, keep_prob):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # C3 input 12 x 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F5 input 4 x 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # F6
        drop5 = tf.nn.dropout(hidden5, keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
        drop6 = tf.nn.dropout(hidden6, keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_biases

    # Training computation.
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [97]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.641101
Minibatch accuracy: 0.0%
Validation accuracy: 12.9%
Minibatch loss at step 100: 1.631075
Minibatch accuracy: 50.0%
Validation accuracy: 47.8%
Minibatch loss at step 200: 1.368231
Minibatch accuracy: 62.5%
Validation accuracy: 66.4%
Minibatch loss at step 300: 0.805215
Minibatch accuracy: 81.2%
Validation accuracy: 73.3%
Minibatch loss at step 400: 0.742243
Minibatch accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss at step 500: 0.525532
Minibatch accuracy: 87.5%
Validation accuracy: 76.5%
Minibatch loss at step 600: 0.982266
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 700: 0.861998
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 800: 0.805645
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 900: 0.584842
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.854802
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%


Well, the accuracy is worst. This net has many meta parameters and I don't feel comfortable in tuning them randomly. I should probably change the depth and make it different between the layers, since it looks like the increasing number of feature maps is a key design item.

I will do so in a next version.

In [131]:
batch_size = 16
patch_size = 5
depth = 16
depth1 = 32
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.4

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth1], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth1]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth1, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
    
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data, keep_prob):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # C3 input 12 x 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F5 input 4 x 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        # F6
        drop5 = tf.nn.dropout(hidden5, keep_prob)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
        drop6 = tf.nn.dropout(hidden6, keep_prob)
        return tf.matmul(drop6, layer5_weights) + layer5_biases

    # Training computation.
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [132]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.477966
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.999182
Minibatch accuracy: 37.5%
Validation accuracy: 39.9%
Minibatch loss at step 200: 1.570479
Minibatch accuracy: 43.8%
Validation accuracy: 58.6%
Minibatch loss at step 300: 1.296772
Minibatch accuracy: 68.8%
Validation accuracy: 62.2%
Minibatch loss at step 400: 1.398355
Minibatch accuracy: 43.8%
Validation accuracy: 72.2%
Minibatch loss at step 500: 0.586528
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Minibatch loss at step 600: 1.056857
Minibatch accuracy: 56.2%
Validation accuracy: 77.4%
Minibatch loss at step 700: 0.719463
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 800: 0.872084
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 900: 0.723940
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 0.539058
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
